In [11]:
import pandas as pd
import rasterio
from rasterio.warp import transform
import numpy as np
import mercantile
import requests
from tqdm import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import multiprocessing as mp
from shapely.geometry import Point
import math

In [2]:
BASE_DIR = Path(r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data")

RAW_DIR = BASE_DIR / "climate_files"        # raw data + DEM cache
WORK_DIR = BASE_DIR / "working_data"        # processed data
DOWNLOAD_DIR = RAW_DIR / "dem_tiles"        # DEM tile cache
INPUT_PKL = WORK_DIR / "clean_labeled_climate_data.pkl"       # your input DataFrame
OUTPUT_PKL = WORK_DIR / "clean_labeled_climate_data_elev.pkl" # output with elevation
ZOOM_LEVEL = 12                             # 30 m resolution
MAX_DOWNLOAD_THREADS = 8                    # concurrent tile downloads
MAX_SAMPLE_PROCESSES = mp.cpu_count()       # all CPU cores

In [3]:
RAW_DIR.mkdir(parents=True, exist_ok=True)
WORK_DIR.mkdir(parents=True, exist_ok=True)
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
df = pd.read_pickle(INPUT_PKL)
print(df.shape)
coords = ['latitude', 'longitude']
df[coords].head()

(470342, 125)


,latitude,longitude
0,49.333333,-95.125000
1,49.333333,-95.083333
2,49.333333,-95.041667
3,49.333333,-95.000000
4,49.333333,-94.958333


In [5]:
def tile_for_point(lat, lon, zoom=ZOOM_LEVEL):
    return mercantile.tile(lon, lat, zoom)

tiles = [tile_for_point(lat, lon) for lat, lon in zip(df.latitude, df.longitude)]
df["tile_x"] = [t.x for t in tiles]
df["tile_y"] = [t.y for t in tiles]
df["tile_z"] = ZOOM_LEVEL
unique_tiles = sorted({(t.x, t.y, t.z) for t in tiles})
print(f"🗺️  Unique tiles required: {len(unique_tiles)}")

🗺️  Unique tiles required: 139383


In [6]:
DEM_DIR = Path(r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\climate_files\dem_tiles")
OUTPUT_PKL = Path(r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\working_data\clean_labeled_climate_data_elev.pkl")
ZOOM_LEVEL = 12

In [7]:
print("Mapping coordinates to DEM tiles...")
tiles = [mercantile.tile(lon, lat, ZOOM_LEVEL) for lat, lon in zip(df.latitude, df.longitude)]
df["tile_x"] = [t.x for t in tiles]
df["tile_y"] = [t.y for t in tiles]
df["tile_z"] = ZOOM_LEVEL
unique_tiles = sorted({(t.x, t.y, t.z) for t in tiles})
print(f"Found {len(unique_tiles)} unique tiles in dataset.")

Mapping coordinates to DEM tiles...
Found 139383 unique tiles in dataset.


In [8]:
tile_path = Path(r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\climate_files\dem_tiles\12_629_1417.tif")

with rasterio.open(tile_path) as src:
    print("Driver:", src.driver)
    print("CRS:", src.crs)
    print("Count (number of bands):", src.count)
    print("Dtype:", src.dtypes)
    print("NoData value:", src.nodata)
    print("Shape:", src.width, "×", src.height)

Driver: GTiff
CRS: EPSG:3857
Count (number of bands): 1
Dtype: ('int16',)
NoData value: -32768.0
Shape: 512 × 512


In [22]:
DEM_DIR = Path(r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\climate_files\dem_tiles")
INPUT_PKL = Path(r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\working_data\clean_labeled_climate_data.pkl")
OUTPUT_PKL = Path(r"C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\working_data\clean_labeled_climate_data_elev.pkl")

MAX_PROCESSES = 4
ZOOM = 12
CHUNK_SIZE = 5000  # max points per parallel batch inside a tile

# Load dataframe
df = pd.read_pickle(INPUT_PKL)
df["elevation"] = np.nan

# Compute tile_x, tile_y
def lonlat_to_tile(lon, lat, z):
    lat_rad = math.radians(lat)
    n = 2 ** z
    xtile = int((lon + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + 1.0 / math.cos(lat_rad)) / math.pi) / 2.0 * n)
    return xtile, ytile

df["tile_x"], df["tile_y"] = zip(*df.apply(lambda r: lonlat_to_tile(r.longitude, r.latitude, ZOOM), axis=1))
df["tile_z"] = ZOOM

# Get unique tiles
unique_tiles = df[['tile_x','tile_y','tile_z']].drop_duplicates().itertuples(index=False, name=None)

# Function to sample a chunk of points in a tile
def sample_points(tile_path, coords, indices):
    # Reproject
    lon, lat = zip(*coords)
    xs, ys = transform("EPSG:4326", "EPSG:3857", lon, lat)
    coords_proj = list(zip(xs, ys))

    try:
        with rasterio.open(tile_path) as src:
            nodata = src.nodata
            results = []
            for idx, val in zip(indices, src.sample(coords_proj)):
                v = float(val[0])
                if v == nodata:
                    v = np.nan
                results.append((idx, v))
        return results
    except Exception as e:
        print(f"⚠️ Error reading {tile_path}: {e}")
        return []

# =========================================================
# PROCESS TILES SEQUENTIALLY
# =========================================================
for tile_key in tqdm(unique_tiles, desc="Processing tiles"):
    x, y, z = tile_key
    tile_path = DEM_DIR / f"{z}_{x}_{y}.tif"
    if not tile_path.exists():
        continue

    subset = df[(df.tile_x == x) & (df.tile_y == y)]
    if subset.empty:
        continue

    coords = list(zip(subset.longitude, subset.latitude))
    indices = subset.index.tolist()

    # Split into chunks for optional parallelization
    chunks = [(coords[i:i+CHUNK_SIZE], indices[i:i+CHUNK_SIZE]) for i in range(0, len(coords), CHUNK_SIZE)]

    results_tile = []
    if len(chunks) == 1:
        # Small tile → process sequentially
        results_tile.extend(sample_points(tile_path, chunks[0][0], chunks[0][1]))
    else:
        # Large tile → parallelize per chunk
        with ProcessPoolExecutor(max_workers=MAX_PROCESSES) as executor:
            futures = [executor.submit(sample_points, tile_path, c[0], c[1]) for c in chunks]
            for f in futures:
                results_tile.extend(f.result())

    # Assign elevations back to DataFrame
    for idx, elev in results_tile:
        df.at[idx, "elevation"] = elev

# Save final DataFrame
df.to_pickle(OUTPUT_PKL)
print(f"✅ Saved DataFrame with elevations: {OUTPUT_PKL}")

Processing tiles: 139383it [2:39:47, 14.54it/s]


✅ Saved DataFrame with elevations: C:\Users\matta\Desktop\Documents\Python\Geolocation\climate_data\working_data\clean_labeled_climate_data_elev.pkl


In [23]:
df.elevation.value_counts()

elevation
 0.0       2647
 2.0       1184
 1.0       1177
 3.0       1037
 5.0        938
           ... 
 4079.0       1
-74.0         1
-27.0         1
-32.0         1
-38.0         1
Name: count, Length: 3912, dtype: int64

In [26]:
df.elevation.max()

np.float64(4255.0)

In [27]:
df.elevation.min()

np.float64(-83.0)